In [82]:
import pandas as pd
import numpy as np

## Joined tables!!

In [85]:
def get_data(label, time, criteria):
    df = pd.read_csv('data/{}/{}/{}.csv'.format(label, time, criteria))
    df['new_gender'] = df.columns[1]
    df['new_height'] = df.columns[3]

    # make the second line as header and drop the first line
    df.columns = df.iloc[0]

    # rename the last column name
    last_column_name = df.columns[-1]
    df = df.rename(columns={last_column_name: "height"})  

    # rename the second last column name
    second_last_column_name = df.columns[-2]
    df = df.rename(columns={second_last_column_name: "gender"})

    # drop the first line
    df = df.drop(df.index[0])
    # print(df.head())
    df.x = df.x.astype(float)
    df.y = df.y.astype(float)
    df.z = df.z.astype(float)
    df.height = df.height.astype(float)

    # rename x,y,z to x_acceleration, y_acceleration, z_acceleration
    df = df.rename(columns={"x": "x_{}".format(criteria), "y": "y_{}".format(criteria), "z": "z_{}".format(criteria)})

    return df

In [89]:
# Get all the folder name under abnormal arm swing
# Path: abnormal_arm_swing
import os
path = 'data/'
labels = os.listdir(path)
# print(labels)
for label in labels:
    if label == '.DS_Store':
        continue
    label_path = path + label
    print(label_path)
    files = os.listdir(label_path)

    for file in files:
        if file == '.DS_Store':
            continue
        print(file)
        df_acc = get_data(label, file, 'accelerometer')
        df_gyro = get_data(label, file, 'gyroscope')
        # if data is empty, skip
        if (df_acc.shape[0] == 0 or df_gyro.shape[0] == 0):
            continue
        # join two tables together, using timestamp as the key
        df = pd.merge(df_acc, df_gyro, on=['timestamp', 'gender', 'height'], how='outer')
        # replace NaN with the previous value
        df_res = df.fillna(method='ffill')
        # if timestamp is the same, only get last row
        df_res = df_res.drop_duplicates(subset=['timestamp'], keep='last')
        # remove the first and last 10 rows
        df_res = df_res.iloc[10:-10]
        
        # save the result to csv
        df_res.to_csv('reformated_data/{}/{}_{}.csv'.format(label, file, 'result'), index=False)
        # print(df_res.describe())
        # print()

data/good_running_posture
1699918813992
1700458989471
1700299659652
1700299748140
1699942129172
1699869318151
1700299839543
1699869406030
1699720588277
1699941957565
1699864647306
1699722124293
1699720726606
1700459077151
1700134452893
1699942037539
1700133420179
1700133334140
1700197635339
1700197423477
1700133336390
data/abnormal_upper_body
1699943036090
1699846694858
1700134049103
1700460211009
1699943258209
1700460369853
1700300807544
1699943133398
1700460495277
1700199618002
1700301084969
1700134978264
1700135063097
1700134167626
1700199265739
1700300995934
1700199518578
1700460583088
1699722315502
1700300891125
1699943488682
data/abnormal_arm_swing
1700198380046
1700299976008
1700198009640
1699943632083
1700197870187
1700051596152
1700134584694
1699943939308
1700300268849
1700459572715
1700459396514
1700300064699
1700051475152
1699943813895
1700198288693
1699943699779
1700300182508
1699718394246
1700133673276
1700459313497
1700134671764
data/abnormal_stride
1700459920901
17001985